# MemoRAG: Enhancing Retrieval-Augmented Generation with Memory Models

## Overview
MemoRAG is a Retrieval-Augmented Generation (RAG) framework that incorporates a memory model as an auxiliary step before the retrieval phase. In doing so, it bridges the gap in contextual understanding and reasoning that standard RAG techniques face when addressing queries with implicit or ambiguous information needs and unstructured external knowledge.

## Motivation
Standard RAG techniques rely heavily on lexical or semantic matching between the query and the knowledge base. While this approach works well for clear question answering tasks with structured knowledge, it often falls short when handling queries with implicit or ambiguous information (e.g., describing the relationships between main characters in a novel) or when the knowledge base is unstructured (e.g., fiction books). In such cases, lexical or semantic matching seldom produces the desired outputs.

## Key Components
1. **Memory**: A compressed representation of the database created by a long-context model, designed to handle and summarize extensive inputs efficiently.
2. **Retriever**: A standard RAG retrieval model responsible for selecting relevant context from the knowledge base to support the generator.
3. **Generator**: A generative language model that produces responses by combining the query with the retrieved context, similar to standard RAG setups.

## Method Details
### 1. Memory
- The memory module serves as an auxiliary component to enhance the retriever’s ability to identify better matches between queries and relevant parts of the database. It takes the original query and the database as inputs and produces staging answers — intermediate outputs like clues, surrogate queries, or key points — which the retriever uses instead of the original query.
- Long-term memory is constructed by running a long-context model, such as Qwen2-7B-Instruct or Mistral-7B-Instruct-v0.2, over the entire database. This process generates a compressed representation of the database through an attention mechanism.
- The compressed representation is stored as key-value pairs, facilitating efficient and accurate retrieval.
- Released memory models include memorag-qwen2-7b-inst and memorag-mistral-7b-inst, derived from Qwen2-7B-Instruct and Mistral-7B-Instruct-v0.2, respectively.

### 2. Retriever
- The retriever is a standard retrieval model, adapted to take processed queries (created by the memory module as staging answers) instead of the original query.
- It outputs the retrieved **context**, which serves as the basis for generating the final answer.


### 3. Generator
- The generator produces the final response by combining the retriever’s output (retrieved context) with the original query.
- MemoRAG ensures compatibility and consistency by using the memory module’s underlying model as the default generator.

## Benefits of the Approach
1. **Extended Scope of Queries:** MemoRAG's preprocessing capabilities enable it to handle complex and long-context tasks that conventional RAG methods struggle with.

2. **Improved Accuracy:** By simplifying and adjusting queries before retrieval, MemoRAG enhances performance over standard RAG methods.

3. **Flexibility:** Adapts to diverse tasks, datasets, and retrieval scenarios.

4. **Robustness:** Improved performance remains consistent across various generators, datasets, and query types.

5. **Efficiency**: The use of key-value compression reduces computational overhead.

## Conclusion
The memory module in MemoRAG significantly enhances comprehension of both the queries and the database, enabling more effective retrieval. Its ability to preprocess queries, generate staging answers, and leverage long-context memory models ensures high-quality responses, making MemoRAG a significant step forward in the evolution of retrieval-augmented generation.


<div style="text-align: center;">

<img src="../images/memo_rag.svg" alt="MemoRAG" style="width:100%; height:auto;">
</div>

## Implementation

### Imports

In [1]:
import os
from typing import Any
from weakref import finalize
from pydantic import BaseModel, Field
from typing import List, Optional
import json
from dotenv import load_dotenv
# from typing import List
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_experimental.graph_transformers.llm import system_prompt
# from langchain_community.vectorstores import FAISS
# from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
from helper_functions import *
from semantic_text_splitter import TextSplitter
from itertools import chain



### OpenAI Setup

In [2]:
load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

### Memory Module Classes

In [3]:
class KeyValuePair(BaseModel):
    topic: str
    details: str

class MemoryResponse(BaseModel):
    pairs: List[KeyValuePair]

In [4]:
class MemoryStore:
    """The MemoryStore class is a realization of the Memory Module discussed in the paper.
    Its 'memorize' method is used to create a prompt cache mimicking a key-value compression of the original text (database).
    This cache can then be used by the 'create_retrieval_prompt' method for creating the processed query to be used later for retrieval"""

    def __init__(self):
        self.embeddings = OpenAIEmbeddings()
        self.store = None
        self.processed_count = 0
        self.json_parse_failures = 0
        self._last_parse_used_fallback = False


    def memorize(self, document: str):
        """Process document into key-value pairs and store them"""

        # self.reset()
        # batch_size = 10
        system_prompt = "Extract key-value pairs from document" ## maybe change this to a better prompt

        kv_cache_prompt = """
        You are provided with a long article, chunk by chunk. Read each chunk carefully and extract key topics and their detailed information.
        For each key topic:
        1. Identify the main concept, entity, or potential question
        2. Provide the corresponding detailed information or answer

        Note: the aim is to mimic kv cache memory creation

        Now, the article begins:
        {document}

        The article ends here.

        RESPONSE FORMAT IS IMPORTANT. You must return a JSON object with the following structure:
        {{
            "pairs": [
                {{
                    "topic": "topic1",
                    "details": "details1"
                }},
                {{
                    "topic": "topic2",
                    "details": "details2"
                }}
            ]
        }}
        """

        print(f"Processing chunk {self.processed_count + 1}...")

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": kv_cache_prompt.format(document=document)}
            ],
            response_format={"type": "json_object"},
            seed=42  # For reproducibility
        )
        # print('*'*20)
        # print(f'memorize response {response}\n\n')
        # Parse the response into key-value pairs
        pairs = self._parse_into_pairs(response.choices[0].message.content)
        if hasattr(self, 'json_parse_failures') and self._last_parse_used_fallback:
            self.json_parse_failures += 1
            if self.json_parse_failures > 0 and self.json_parse_failures % 5 == 0:
               print(f"Warning: JSON parsing has failed {self.json_parse_failures} times")

        print('*'*20)
        print(f'first extracted pairs: {pairs[0:3]}\n\n')

        # Batch process pairs
        texts = []
        metadatas = []

        for topic, details in pairs:
            if not topic or not details:  # Skip empty pairs
                continue

            combined_text = f"Topic: {topic}\nDetails: {details}"
            texts.append(combined_text)
            metadatas.append({"topic": topic})
        print('*'*20)
        print(f'num of extracted texts: {len(texts)}\n\n')
        if self.store is None:
            self.store = FAISS.from_texts(texts, self.embeddings, metadatas=metadatas)
        else:
            existing_docs = self.store.similarity_search("")
            existing_topics = {doc.metadata.get("topic") for doc in existing_docs}

            # Filter out duplicates
            new_texts = []
            new_metadatas = []
            for text, metadata in zip(texts, metadatas):
                if metadata["topic"] not in existing_topics:
                    new_texts.append(text)
                    new_metadatas.append(metadata)

            if new_texts:
                self.store.add_texts(new_texts, metadatas=new_metadatas)

        self.processed_count += 1
        print(f"Processed {self.processed_count} chunks so far\n\n")

    def _parse_into_pairs(self, content: str):
        self._last_parse_used_fallback = False
        try:
            # Parse JSON and validate with Pydantic
            data = json.loads(content)
            validated_data = MemoryResponse(**data)
            return [(pair.topic, pair.details) for pair in validated_data.pairs]
        except (json.JSONDecodeError, ValueError) as e:
            print(f"Error parsing response: {e}")
            # Fall back to text parsing if JSON fails
            self._last_parse_used_fallback = True  # Set this to True
            return self._parse_text_into_pairs(content)

    def _parse_text_into_pairs(self, text: str):
        """Original text parsing method as fallback"""
        pairs = []
        lines = text.split('\n')
        current_topic = None
        current_details = []

        for line in lines:
            if line.startswith('Topic:'):
                if current_topic:  # Save previous pair
                    pairs.append((current_topic, ' '.join(current_details)))
                current_topic = line[6:].strip()
                current_details = []
            elif line.startswith('Details:'):
                current_details.append(line[8:].strip())
            elif current_details:  # Continue adding to details if already started
                current_details.append(line.strip())

        # Add the last pair
        if current_topic:
            pairs.append((current_topic, ' '.join(current_details)))

        return pairs

    def create_retrieval_queries(self, query: str) -> str | None:
        """Generate staging answers y = Θ_mem(q, D | θ_mem)
        This should provide rough clues/outline to guide context retrieval"""

        if not self.store:
            return None

        results = self.store.similarity_search_with_score(query, k=min(10, self.store.index.ntotal)) # consider increasing k to more than 10. maybe try even top 30% or something
        relevant_info = [
            f"Topic: {doc.metadata['topic']}\nDetails: {doc.page_content}"
            for doc, _ in results
        ]

        memorag_span_prompt = """
            You are given a question related to an article. To answer it effectively, you need to use specific details from the article. You are not provided with the whole article. Instead, you are provided with specific and relevant information from it.
             Your task is to identify and extract one or more specific clue texts from the provided information that are relevant to the question.

            ### Question: {question}
            ### Information: {relevant_info}
            ### Instructions:
            1. You have a general understanding of the provided information. Your task is to generate one or more specific clues that will help in searching for supporting evidence within the article.
            2. The clues are in the form of text spans that will assist in answering the question.
            3. Only output the clues. If there are multiple clues, separate them with a newline.
            """
        memorag_sur_prompt = """
            You are given a question related to an article. To answer it effectively, you need to use specific details from the article.  You are not provided with the whole article. Instead, you are provided with specific and relevant information from it.
            Your task is to generate precise clue questions that can help locate the necessary information for answering the question.

            ### Question: {question}
            ### Information: {relevant_info}
            ### Instructions:
            1. You have a general understanding of the provided information. Your task is to generate one or more specific clues that will help in searching for supporting evidence within the article.
            2. The clues are in the form of precise surrogate questions that clarify the original question.
            3. Only output the clues. If there are multiple clues, separate them with a newline.
            """

        system_prompt = "You are an expert in information retrieval. Your task is to extract specific and relevant information from the provided context to help answer the given question."

        text_spans = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": memorag_span_prompt.format(question=query, relevant_info = relevant_info)}
            ]
        ).choices[0].message.content
        surrogate_queries = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": memorag_sur_prompt.format(question=query, relevant_info = relevant_info)}
            ],
            temperature=0.5  # Allow for some creativity in generating clues
        ).choices[0].message.content

        retrieval_query = text_spans.split("\n") + surrogate_queries.split("\n")
        # print(20*'*')
        # print(f'retrieval query all: {retrieval_query}\n\n')
        retrieval_query = [q for q in retrieval_query if len(q.split()) > 3]
        # print(20*'*')
        # print(f'retrieval query no shorts: {retrieval_query}\n\n')
        retrieval_query.append(query)
        # print(20*'*')
        # print(f'retrieval query including original query: {retrieval_query}\n\n')

        return retrieval_query

    def save_store(self, path: str):
        if self.store:
            self.store.save_local(path)

    def load_store(self, path: str):
        self.store = FAISS.load_local(path, self.embeddings, allow_dangerous_deserialization=True)

### Retrieval Function

In [5]:
def retrieve_context(retrieval_query: str, vectorstore, k: int = 5) -> List[str]:
    """Retrieve relevant context using the staging answer and the database vectorstore.
    Implements c = Γ(y, D | γ) from the paper"""

    results = vectorstore.similarity_search(retrieval_query, k=k)
    contexts = [doc.page_content for doc in results]

    return contexts

### Generation Function

In [6]:
def generate_answer(query: str, contexts: List[str], temperature: float = 0) -> str:
    """Generate final answer y = Θ(q, c | θ)"""
    prompt = f"""Based ONLY on the provided context, answer the query.

Query: {query}

Context:
{' '.join(contexts)}

Provide a clear and concise answer based solely on the information in the context. If the context doesn't contain sufficient information to fully answer the query, clearly state that the necessary information is not available in the provided context rather than using any external knowledge.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert in generating answers from given text. Provide clear, concise answers."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=500
    )

    return response.choices[0].message.content

### Query Processing Function

In [7]:
def process_query(query: str, memory_store, vectorstore):
    print('*'*20)
    print("\nProcessing Query:", query)
    print("=" * 50)

    # y = Θ_mem(q, D | θ_mem)
    retrieval_queries = memory_store.create_retrieval_queries(query)
    print('*'*20)
    print(f"\n\nRetrieval Queries:\n{retrieval_queries}\n\n")

    all_contexts = []
    for retrieval_query in retrieval_queries:
        contexts = retrieve_context(retrieval_query, vectorstore, k=3)  # Smaller k per query
        all_contexts.extend(contexts)
    # Remove duplicates
    unique_contexts = list(dict.fromkeys(all_contexts))  # Preserves order

    # c = Γ(y, D | γ)
    all_contexts = []
    for retrieval_query in retrieval_queries:
        contexts = retrieve_context(retrieval_query, vectorstore, k=3)  # Smaller k per query
        all_contexts.extend(contexts)
    # Remove duplicates
    unique_contexts = list(dict.fromkeys(all_contexts))  # Preserves order
    # contexts = [retrieve_context(retrieval_query, vectorstore) for retrieval_query in retrieval_queries][0]
    print('*'*20)
    print(f"Retrieved Context Example: {unique_contexts[0]}\n\n")

    # y = Θ(q, c | θ)
    final_answer = generate_answer(query, unique_contexts)
    print('*'*20)
    print(f"\nFinal Answer: {final_answer}")

    return unique_contexts, final_answer

### Initialize Components

In [8]:
# Initialize memory store
memory_store = MemoryStore()

# Load and process document
path = "../data/Understanding_Climate_Change.pdf"
loader = PyPDFLoader(path)
documents = loader.load()
document_text = '\n'.join([doc.page_content for doc in documents])
memory_store.memorize(document_text)
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

Processing chunk 1...
********************
memorize response ChatCompletion(id='chatcmpl-B7eSMIgeiSfp81y2N2YPQvcFsbBVs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "pairs": [\n        {\n            "topic": "Understanding Climate Change",\n            "details": "Climate change refers to significant, long-term changes in the global climate, including temperature, precipitation, and wind patterns, primarily driven by human activities like burning fossil fuels and deforestation."\n        },\n        {\n            "topic": "Historical Context",\n            "details": "The Earth\'s climate has changed throughout history with seven cycles of glacial advance and retreat over 650,000 years. The last ice age ended about 11,700 years ago, leading to the Holocene epoch where human civilizations flourished, but modern industrial activities have induced unprecedented climate changes."\n        },\n        {\n            "topic": 

### Usage Examples

In [9]:
query_1 = "What are the impacts of climate change on biodiversity?"
query_2 = "Please summarize the climate change article"
query_3 = "Describe the social and economic influence of climate change."

for query in [query_1, query_2, query_3]:
    process_query(query, memory_store, chunks_vector_store)


********************

Processing Query: What are the impacts of climate change on biodiversity?
********************


Retrieval Queries:
['- Climate change impacts ecosystems by altering species distributions and threatening biodiversity.', '- These impacts threaten ecosystems, human health, and food security.', 'What specific ways does climate change alter species distributions?  ', 'How does climate change threaten biodiversity and what conservation strategies are necessary?  ', 'What are the broader ecological impacts of climate change on ecosystems?  ', 'What role does biodiversity play in climate action strategies?', 'What are the impacts of climate change on biodiversity?']


********************
Retrieved Context Example: Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shifts in plant and animal species composition. These changes  

## Comparison